# Load Packages

In [19]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [20]:
from nemo_microservices import NeMoMicroservices
from nemo_microservices.beta.data_designer import (
    DataDesignerConfigBuilder,
    DataDesignerClient,
)
from nemo_microservices.beta.data_designer.config import columns as C
from nemo_microservices.beta.data_designer.config import params as P

ndd = DataDesignerClient(
    client=NeMoMicroservices(
            base_url="https://ai.api.nvidia.com/v1/nemo/dd",
            default_headers={"Authorization": f"Bearer {os.getenv('NVIDIA_API_KEY')}"}
    )
)
# The following model aliases are availble by default in this hosted Data Designer
# nemotron-nano-v2, nemotron-super, mistral-small, gpt-oss-20b, gpt-oss-120b, llama-4-scout-17b  
alias="nemotron-nano-v2"

config_builder = DataDesignerConfigBuilder()

In [21]:
config_builder.add_column(
    C.SamplerColumn(
        name="product_category",
        type=P.SamplerType.CATEGORY,
        params=P.CategorySamplerParams(
            values=[
                "Electronics",
                "Clothing",
                "Home & Kitchen",
                "Books",
                "Home Office",
            ],
        ),
    )
)

config_builder.add_column(
    C.SamplerColumn(
        name="product_subcategory",
        type=P.SamplerType.SUBCATEGORY,
        params=P.SubcategorySamplerParams(
            category="product_category",
            values={
                "Electronics": [
                    "Smartphones",
                    "Laptops",
                    "Headphones",
                    "Cameras",
                    "Accessories",
                ],
                "Clothing": [
                    "Men's Clothing",
                    "Women's Clothing",
                    "Winter Coats",
                    "Activewear",
                    "Accessories",
                ],
                "Home & Kitchen": [
                    "Appliances",
                    "Cookware",
                    "Furniture",
                    "Decor",
                    "Organization",
                ],
                "Books": [
                    "Fiction",
                    "Non-Fiction",
                    "Self-Help",
                    "Textbooks",
                    "Classics",
                ],
                "Home Office": [
                    "Desks",
                    "Chairs",
                    "Storage",
                    "Office Supplies",
                    "Lighting",
                ],
            },
        ),
    )
)

config_builder.add_column(
    C.SamplerColumn(
        name="target_age_range",
        type=P.SamplerType.CATEGORY,
        params=P.CategorySamplerParams(
            values=["18-25", "25-35", "35-50", "50-65", "65+"]
        ),
    )
)

# This column will sample synthetic person data based on statistics from the US Census.
config_builder.add_column(
    C.SamplerColumn(
        name="customer",
        type=P.SamplerType.PERSON,
        params=P.PersonSamplerParams(age_range=[18, 70]),
    )
)

config_builder.add_column(
    C.SamplerColumn(
        name="number_of_stars",
        type=P.SamplerType.UNIFORM,
        params=P.UniformSamplerParams(low=1, high=5),
        convert_to="int",
    )
)

config_builder.add_column(
    C.SamplerColumn(
        name="review_style",
        type=P.SamplerType.CATEGORY,
        params=P.CategorySamplerParams(
            values=["rambling", "brief", "detailed", "structured with bullet points"],
            weights=[1, 2, 2, 1],
        ),
    )
)

DataDesignerConfigBuilder(
    sampler_columns: [
        "product_category",
        "product_subcategory",
        "target_age_range",
        "customer",
        "number_of_stars",
        "review_style"
    ]
)

In [22]:
config_builder.add_column(
    C.LLMTextColumn(
        name="product_name",
        prompt=(
            "Come up with a creative product name for a product in the '{{ product_category }}' category, focusing "
            "on products related to '{{ product_subcategory }}'. The target age range of the ideal customer is "
            "{{ target_age_range }} years old. Respond with only the product name, no other text."
        ),
        model_alias=alias,
    )
)

config_builder.add_column(
    C.LLMTextColumn(
        name="customer_review",
        prompt=(
            "You are a customer named {{ customer.first_name }} from {{ customer.city }}, {{ customer.state }}. "
            "You are {{ customer.age }} years old and recently purchased a product called {{ product_name }}. "
            "Write a review of this product, which you gave a rating of {{ number_of_stars }} stars. "
            "The style of the review should be '{{ review_style }}'. "
        ),
        model_alias=alias,
    )
)

DataDesignerConfigBuilder(
    sampler_columns: [
        "product_category",
        "product_subcategory",
        "target_age_range",
        "customer",
        "number_of_stars",
        "review_style"
    ]
    llm_text_columns: ['product_name', 'customer_review']
)

In [23]:
preview = ndd.preview(config_builder, num_records=10, verbose_logging=True)

[13:31:15] [INFO] 🎲 Step 1: Using samplers to generate 6 columns
[13:31:15] [INFO]   |-- 🎲 🧑‍🎨 Initializing person generation
[13:31:15] [INFO]   |-- 🎲 Using numerical samplers to generate 10 records across 6 columns
[13:31:16] [INFO] 🦜 Step 2: Generating text column `product_name`
[13:31:16] [INFO]   |-- 📝 Preparing template to generate data column `product_name`
[13:31:16] [INFO]   |   |-- model_alias: nemotron-nano-v2
[13:31:16] [INFO]   |-- 🩺 Running health check for model with alias 'nemotron-nano-v2'
[13:31:16] [INFO]   |-- ✅ Health check completed for model with alias 'nemotron-nano-v2'
[13:31:16] [INFO]   |-- Model config being used for model alias 'nemotron-nano-v2': 
{
    "alias": "nemotron-nano-v2",
    "model": {
        "api_endpoint": {
            "url": "https://integrate.api.nvidia.com/v1",
            "model_id": "nvidia/nvidia-nemotron-nano-9b-v2",
            "provider_type": "openai"
        }
    },
    "inference_parameters": {
        "temperature": 0.6,
      

In [24]:
preview.display_sample_record()

                                                 Generated Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name                ┃ Value                                                                                     ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ product_category    │ Books                                                                                     │
├─────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────┤
│ product_subcategory │ Self-Help                                                                                 │
├─────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────┤
│ target_age_range    │ 65+                                                                                       │
├─────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────┤
│ customer            │ {                                                                                         │
│                     │     'first_name': 'Ariel',                                                                │
│                     │     'middle_name': '',                                                                    │
│                     │     'last_name': 'Preim',                                                                 │
│                     │     'sex': 'Female',                                                                      │
│                     │     'age': 68,                                                                            │
│                     │     'zipcode': '01930',                                                                   │
│                     │     'street_number': 52,                                                                  │
│                     │     'street_name': 'Retreat Ave',                                                         │
│                     │     'unit': '',                                                                           │
│                     │     'city': 'Gloucester',                                                                 │
│                     │     'state': 'MA',                                                                        │
│                     │     'county': 'Essex County',                                                             │
│                     │     'country': 'USA',                                                                     │
│                     │     'ethnic_background': 'white',                                                         │
│                     │     'marital_status': 'married_present',                                                  │
│                     │     'education_level': 'high_school',                                                     │
│                     │     'bachelors_field': 'no degree',                                                       │
│                     │     'occupation': 'registered_nurse',                                                     │
│                     │     'uuid': '61da2d27-28aa-4c55-91e5-171549e3290d',                                       │
│                     │     'locale': 'en_US',                                                                    │
│                     │     'phone_number': '351-270-6557',                                                       │
│                     │     'email_address': 'arielpreim@gmail.com',                                              │
│                     │     'birth_date': '1956-10-26',                                                           │
│                     │     'ssn': '025-86-6760'        

In [25]:
preview.dataset

,product_category,product_subcategory,target_age_range,customer,number_of_stars,review_style,product_name,product_name__reasoning_trace,customer_review,customer_review__reasoning_trace
0,Books,Self-Help,65+,"{'first_name': 'Ariel', 'middle_name': '', 'la...",2,brief,Golden Steps,"Okay, the user wants a creative product name f...",**2 stars** \nGolden Steps didn’t meet expect...,"Okay, let's see. The user wants me to write a ..."
1,Clothing,Activewear,50-65,"{'first_name': 'Sally', 'middle_name': 'K', 'l...",4,brief,Zenith Vitality Legacy,"Okay, the user wants a creative product name f...",**Review:** Great product overall! Noticed inc...,"Okay, Sally from LA, 34, bought Zenith Vitalit..."
2,Books,Fiction,35-50,"{'first_name': 'Madelyn', 'middle_name': 'S', ...",4,structured with bullet points,The Veil of Forgotten Tales,"Okay, the user wants a creative product name f...",- **Engaging Storytelling:** The Veil of Forgo...,"Okay, let's see. The user wants a product revi..."
3,Electronics,Smartphones,65+,"{'first_name': 'Isabel', 'middle_name': '', 'l...",1,detailed,GoldenTouch,"Okay, the user wants a creative product name f...",**Review Title: A Disappointing Waste of Time ...,"Okay, Isabel is a 69-year-old customer from Ph..."
4,Clothing,Activewear,25-35,"{'first_name': 'Darby', 'middle_name': 'E', 'l...",2,structured with bullet points,VeloCore,"Okay, the user wants a creative product name f...",- **Disappointing Performance**: The VeloCore ...,"Okay, the user wants me to write a structured ..."
5,Clothing,Women's Clothing,35-50,"{'first_name': 'Salvatore', 'middle_name': 'Pe...",2,brief,NexaLuxe,"Okay, the user wants a creative product name f...","**Salvatore (Antioch, IL):** NexaLuxe underdel...","Okay, let's tackle this query. The user wants ..."
6,Books,Classics,35-50,"{'first_name': 'Long', 'middle_name': 'B', 'la...",2,detailed,The Timeless Shelf,"Okay, the user wants a creative product name f...",**Review Title: Disappointing Quality and Poor...,"Okay, I need to write a detailed review for Th..."
7,Home Office,Office Supplies,25-35,"{'first_name': 'Duwayne', 'middle_name': 'Char...",3,rambling,NestHub,"Okay, the user wants a creative product name f...","**Title: NestHub: Okay, I Guess? Not What I Ex...","Okay, I need to write a 3-star review for the ..."
8,Home Office,Desks,18-25,"{'first_name': 'April', 'middle_name': 'Madeli...",2,detailed,NexaDesk,"Okay, the user wants a creative product name f...",**Title: Disappointing Experience with NexaDes...,"Okay, let's tackle this query. The user wants ..."
9,Home & Kitchen,Furniture,35-50,"{'first_name': 'Jeffery', 'middle_name': 'L', ...",3,rambling,HavenNest,"Okay, the user wants a creative product name f...","Well, you know, I guess I should start by sayi...","Okay, I need to write a 3-star review for Have..."
